<a href="https://colab.research.google.com/github/vnavya2004/BTP/blob/main/semi_supervised_spanish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from tqdm import tqdm
from google.colab import files
import torch.nn.functional as F
from transformers import AutoModelForSequenceClassification

# Load the XLM-RoBERTa tokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# Assuming you're using Google Colab and uploaded a file
uploaded = files.upload()

# Read the CSV file
df = pd.read_csv("Book3.csv", encoding='latin1')
df = df.sample(frac=1, random_state=42)  # Shuffle the dataset


Saving Book3.csv to Book3 (2).csv


In [4]:
df.head()

,ï»¿traducido,class
0,AYUDI necesita encontrar una salida a esta tri...,1
17,La relaciÃ³n abusiva me estÃ¡ matando.por favo...,1
15,Helpi simplemente se tragÃ³ un puÃ±ado de mi a...,1
1,Al ver destellos de suicidio de autoshamiento ...,1
8,"Esto suena absolutamente desagradable, egoÃ­st...",1


In [10]:

# Specify the columns for features (tweets) and labels
tweets_column = 'ï»¿traducido'
labels_column = 'class'
NUM_LABELS = len(df[labels_column].unique())

# Split the dataset
labeled_df, unlabeled_df = train_test_split(df, test_size=0.6, stratify=df[labels_column])
unlabeled_df, test_df = train_test_split(unlabeled_df, test_size=1/3, stratify=unlabeled_df[labels_column])

# Tokenize and encode the data
def encode_data(df, column):
    encoded_data = tokenizer.batch_encode_plus(
        df[column].tolist(),
        add_special_tokens=True,
        return_attention_mask=True,
        pad_to_max_length=True,
        max_length=256,
        return_tensors='pt'
    )
    input_ids = encoded_data['input_ids']
    attention_masks = encoded_data['attention_mask']
    return input_ids, attention_masks

input_ids_labeled, attention_masks_labeled = encode_data(labeled_df, tweets_column)
labels_labeled = torch.tensor(labeled_df[labels_column].values, dtype=torch.float)

input_ids_unlabeled, attention_masks_unlabeled = encode_data(unlabeled_df, tweets_column)
input_ids_test, attention_masks_test = encode_data(test_df, tweets_column)
labels_test = torch.tensor(test_df[labels_column].values, dtype=torch.float)

dataset_labeled = TensorDataset(input_ids_labeled, attention_masks_labeled, labels_labeled)
dataset_unlabeled = TensorDataset(input_ids_unlabeled, attention_masks_unlabeled)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

# Define the XLM-RoBERTa model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=NUM_LABELS)

# Set up the device for training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Set up the optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8)
epochs = 5
batch_size = 32
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(dataset_labeled)*epochs)

# Training loop
def binary_accuracy(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return accuracy_score(labels_flat, preds_flat)

def binary_f1_score(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def binary_precision(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return precision_score(labels_flat, preds_flat, average='weighted')

def binary_recall(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return recall_score(labels_flat, preds_flat, average='weighted')

def train_model(model, dataset, optimizer, scheduler, epochs):
    for epoch in range(1, epochs + 1):
        model.train()
        loss_train_total = 0
        progress_bar = tqdm(DataLoader(dataset, sampler=RandomSampler(dataset), batch_size=batch_size), desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
        for batch in progress_bar:
            model.zero_grad()
            batch = tuple(b.to(device) for b in batch)
            inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
            outputs = model(**inputs)
            loss = outputs[0]
            loss_train_total += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item() / len(batch))})

        loss_train_avg = loss_train_total / len(dataset)
        tqdm.write(f'\nEpoch {epoch}')
        tqdm.write(f'Training loss: {loss_train_avg}')

# Train the model on labeled data
train_model(model, dataset_labeled, optimizer, scheduler, epochs)

# Predict labels for unlabeled data
model.eval()
predicted_labels = []

for batch in tqdm(DataLoader(dataset_unlabeled, batch_size=batch_size), desc='Predicting labels for unlabeled data', leave=False, disable=False):
    batch = tuple(b.to(device) for b in batch)
    inputs = {'input_ids': batch[0], 'attention_mask': batch[1]}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs[0]
    predictions = torch.argmax(logits, dim=1).cpu().numpy()
    predicted_labels.extend(predictions)

predicted_labels = torch.tensor(predicted_labels)

# Create a new dataset with original labeled data and newly labeled data
combined_input_ids = torch.cat((input_ids_labeled, input_ids_unlabeled), dim=0)
combined_attention_masks = torch.cat((attention_masks_labeled, attention_masks_unlabeled), dim=0)
combined_labels = torch.cat((labels_labeled, predicted_labels), dim=0)

dataset_combined = TensorDataset(combined_input_ids, combined_attention_masks, combined_labels)

# Train the model on the combined dataset
optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(dataset_combined)*epochs)
train_model(model, dataset_combined, optimizer, scheduler, epochs)

# Evaluate the model on the test data
model.eval()
predictions, true_vals = [], []

for batch in tqdm(DataLoader(dataset_test, batch_size=batch_size), desc='Evaluating', leave=False, disable=False):
    batch = tuple(b.to(device) for b in batch)
    inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs[1]
    logits = logits.detach().cpu().numpy()
    label_ids = inputs['labels'].cpu().numpy()
    predictions.append(logits)
    true_vals.append(label_ids)

predictions = np.concatenate(predictions, axis=0)
true_vals = np.concatenate(true_vals, axis=0)

test_accuracy = binary_accuracy(predictions, true_vals)
test_f1 = binary_f1_score(predictions, true_vals)
test_precision = binary_precision(predictions, true_vals)
test_recall = binary_recall(predictions, true_vals)

# Print out the evaluation metrics on test data
print(f'Testing Accuracy: {test_accuracy}')
print(f'Testing F1 Score: {test_f1}')
print(f'Testing Precision: {test_precision}')
print(f'Testing Recall: {test_recall}')


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2760: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint 


Epoch 1
Training loss: 0.2911902964115143



Epoch 2
Training loss: 0.2559983357787132



Epoch 1
Training loss: 0.11046432051807642



Epoch 2
Training loss: 0.08993554953485727


Testing Accuracy: 0.0
Testing F1 Score: 0.0
Testing Precision: 0.0
Testing Recall: 0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
